# Libraries

In [30]:
# pyTorch
import torch
from torch import nn
from torch.optim import Adam
from torch.nn.functional import softmax
import torch.optim as optim
from torch.nn import MSELoss
import torch.nn.functional as F

# numpy
import numpy as np

# matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# scikit-learn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer

# pandas
import pandas as pd

# pickle
import pickle

# math
import math

# random
import random


# Récupération des données

In [2]:
with open('movie_ratings_500_id.pkl', 'rb') as file:
    data = pickle.load(file)

#On récupère les noms des films (les noms des films sont uniques car ce sont des clés de dictionnaires)
liste_des_films = list(data.keys())
nb_films = len(liste_des_films)

print("Le nb de films : ",nb_films)

Le nb de films :  528


On vérifie que chaque utilisateur a noté au plus une fois chaque film

In [3]:
check = True
for film in liste_des_films:
    
    unique_user_id = set()
    for elem in  data[film]:
        unique_user_id.add(elem['user_id'])
        
    if len(unique_user_id) < len(data[film]):
        check = False       
if check :     
    print("Chaque utilisateur a noté au plus une fois chaque film")

Chaque utilisateur a noté au plus une fois chaque film


On récupère les identifiants des utilisateurs 

In [4]:
liste_user_id = set() #on utilise un set pour ne pas avoir de doublons 

for film in liste_des_films:
    for elem in  data[film]:
        liste_user_id.add(elem['user_id'])
        
liste_user_id = list(liste_user_id)
nb_user = len(liste_user_id)
print("Le nb d'utilisateurs : ", nb_user)

df = pd.DataFrame(index=liste_user_id, columns=liste_des_films)

for film in liste_des_films:
    for elem in data[film]:
        user_id = elem['user_id']
        note = int(elem['user_rating'])  
        df.at[user_id, film] = note

Le nb d'utilisateurs :  36968


Si l'utilisateur n'a pas noté le film, on met la note à -1.

In [5]:
df.fillna(-1, inplace=True) 
nb_note_manquante = (df.eq(-1)).sum().sum()
nombre_user_ayant_note_aucun_film = df[df.eq(-1).all(axis=1)].shape[0]
print(nombre_user_ayant_note_aucun_film) 

0


Tous les utilisateurs ont noté au moins un film

On récupère les informations supplémentaires sur les films 


In [6]:
with open('movie_metadata.pkl', 'rb') as file:
    data1 = pickle.load(file)

#On crée un dataframe avec 5 colonnes ('film', 'directeur', 'genre', 'acteurs', 'titre')

df1 = pd.DataFrame.from_dict(data1, orient='index')
df1.reset_index(inplace=True)
df1.columns = ['film', 'directeur', 'genres', 'acteurs', 'titre']

df1

,film,directeur,genres,acteurs,titre
0,tt0305224,Peter Segal,[Comedy],"[Jack Nicholson, Adam Sandler, Marisa Tomei, W...",Anger Management
1,tt0245046,Gillian Armstrong,"[Drama, Romance, Thriller]","[Cate Blanchett, James Fleet, Abigail Cruttenden]",Charlotte Gray
2,tt0185125,Pedro Almodóvar,[Drama],"[Cecilia Roth, Marisa Paredes, Candela Peña, P...",All About My Mother
3,tt0196229,Ben Stiller,[Comedy],"[Ben Stiller, Owen Wilson, Christine Taylor, W...",Zoolander
4,tt0308644,Marc Forster,"[Biography, Drama, Family]","[Johnny Depp, Kate Winslet, Julie Christie, Du...",Finding Neverland
...,...,...,...,...,...
523,tt0203019,George Tillman Jr.,"[Biography, Drama]","[Cuba Gooding Jr., Robert De Niro, Charlize Th...",Men of Honor
524,tt0169547,Sam Mendes,"[Drama, Romance]","[Kevin Spacey, Annette Bening, Thora Birch, We...",American Beauty
525,tt0227538,Robert Rodriguez,"[Action, Adventure, Comedy]","[Alexa PenaVega, Daryl Sabara, Antonio Banderas]",Spy Kids
526,tt0374536,Nora Ephron,"[Comedy, Fantasy, Romance]","[Nicole Kidman, Will Ferrell, Shirley MacLaine...",Bewitched


In [7]:
nb_genre_different_pour_un_meme_film = set()
nb_acteurs_different_pour_un_meme_film = set()

for i in range(528):
    nb_genre_different_pour_un_meme_film.add(len(df1.iloc[i][2]))
    nb_acteurs_different_pour_un_meme_film.add(len(df1.iloc[i][3]))
    
print(list(nb_genre_different_pour_un_meme_film))
print(list(nb_acteurs_different_pour_un_meme_film))

[1, 2, 3]
[2, 3, 4, 5, 6, 7, 8, 10, 11]


# Réseaux de neurones

## Traitement des données

On va créér un dataframe avec 3 colonnes (user_id,film,note)

In [8]:
# On réinitialise l'index pour obtenir une colonne 'user_id'
df_pour_reseau = df.copy()
df_pour_reseau.reset_index(inplace=True)
df_pour_reseau.rename(columns={'index': 'user_id'}, inplace=True)

# On convertit le DataFrame 
df_pour_reseau = pd.melt(df_pour_reseau, id_vars=['user_id'], var_name='film', value_name='note')

# On ne garde que les notes différente de -1 càd les notes représentant un visionage 
df_pour_reseau = df_pour_reseau[df_pour_reseau['note'] != -1]

df_pour_reseau = df_pour_reseau.reset_index(drop=True)

df_pour_reseau.head()

,user_id,film,note
0,946314,tt0305224,3
1,1825313,tt0305224,3
2,1437190,tt0305224,4
3,1857486,tt0305224,4
4,994483,tt0305224,3


On veut aussi prendre en compte les informations supplémentaires sur les films
Pour cela, on va faire une jointure entre df_pour_reseau et df1 sur la colonne 'film'

In [9]:
df_pour_reseau = pd.merge(df_pour_reseau, df1, on='film', how='inner')
df_pour_reseau.head()

,user_id,film,note,directeur,genres,acteurs,titre
0,946314,tt0305224,3,Peter Segal,[Comedy],"[Jack Nicholson, Adam Sandler, Marisa Tomei, W...",Anger Management
1,1825313,tt0305224,3,Peter Segal,[Comedy],"[Jack Nicholson, Adam Sandler, Marisa Tomei, W...",Anger Management
2,1437190,tt0305224,4,Peter Segal,[Comedy],"[Jack Nicholson, Adam Sandler, Marisa Tomei, W...",Anger Management
3,1857486,tt0305224,4,Peter Segal,[Comedy],"[Jack Nicholson, Adam Sandler, Marisa Tomei, W...",Anger Management
4,994483,tt0305224,3,Peter Segal,[Comedy],"[Jack Nicholson, Adam Sandler, Marisa Tomei, W...",Anger Management


Les valeurs contenues dans les colonnes genres et acteurs sont des listes de valeurs.  
Pour faciliter le traitement avec les réseaux de neurones :  
on attribue 1 si le film appartient à un genre et 0 sinon.  
On fait pareil pour les acteurs. 

In [10]:
mlbg = MultiLabelBinarizer()
genres_encoded = mlbg.fit_transform(df_pour_reseau['genres'])
df_genres = pd.DataFrame(genres_encoded, columns=mlbg.classes_)

mlba = MultiLabelBinarizer()
acteurs_encoded = mlba.fit_transform(df_pour_reseau['acteurs'])
df_acteurs = pd.DataFrame(acteurs_encoded, columns=mlba.classes_)

liste_genres = mlbg.classes_
liste_acteurs = mlba.classes_

df_genres.head()

,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


On concatène les DataFrames encodés avec le DataFrame d'origine

In [11]:
df_pour_reseau = pd.concat([df_pour_reseau, df_genres, df_acteurs], axis=1)

#On supprimer les colonnes originales 'genres' et 'acteurs' car on en a plus besoin
df_pour_reseau.drop(['genres', 'acteurs'], axis=1, inplace=True)

#On met la colonne note à la fin du dataframe 
colonne_note = df_pour_reseau.pop('note')
df_pour_reseau['note'] = colonne_note

df_pour_reseau.head()

,user_id,film,directeur,titre,Action,Adventure,Animation,Biography,Comedy,Crime,...,William Hurt,William Shatner,Winona Ryder,Wood Harris,Woody Allen,Woody Harrelson,Yoshihiko Hakamada,Yun-Fat Chow,Zach Braff,note
0,946314,tt0305224,Peter Segal,Anger Management,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,3
1,1825313,tt0305224,Peter Segal,Anger Management,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,3
2,1437190,tt0305224,Peter Segal,Anger Management,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,4
3,1857486,tt0305224,Peter Segal,Anger Management,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,4
4,994483,tt0305224,Peter Segal,Anger Management,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,3


Les notes possibles sont 1,2,3,4 et 5

In [12]:
notes_differentes_possibles = df_pour_reseau["note"].unique()
notes_differentes_possibles 

array([3, 4, 5, 1, 2])

On récupère dans le dataframe la ligne qui corresponds à l'user_id = '1380819'  
on ne garde que les films qu'il n'a pas vu càd note = -1 


In [13]:
### Données pour prédire
ligne_user = df.loc[df.index == '1380819']

ligne_user = ligne_user.where(ligne_user == -1)
ligne_user = ligne_user.dropna(axis=1, how='any')

#on créer un dataframe avec 2 colonnes (user_id,film)
ligne_user = ligne_user.reset_index()
ligne_user = ligne_user.rename(columns={'index': 'user_id'})
df_user = pd.melt(ligne_user, id_vars=['user_id'], var_name='film', value_name='note')
df_user = df_user.drop(columns=['note'])

### Les informations supplémentaires sur les films

Pour cela, on va faire une jointure entre df_user et df1 sur la colonne 'film'

In [14]:
df_user = pd.merge(df_user, df1, on='film', how='inner')
df_user.head()

,user_id,film,directeur,genres,acteurs,titre
0,1380819,tt0245046,Gillian Armstrong,"[Drama, Romance, Thriller]","[Cate Blanchett, James Fleet, Abigail Cruttenden]",Charlotte Gray
1,1380819,tt0185125,Pedro Almodóvar,[Drama],"[Cecilia Roth, Marisa Paredes, Candela Peña, P...",All About My Mother
2,1380819,tt0308644,Marc Forster,"[Biography, Drama, Family]","[Johnny Depp, Kate Winslet, Julie Christie, Du...",Finding Neverland
3,1380819,tt0282687,Stephen Herek,"[Comedy, Romance]","[Angelina Jolie, Edward Burns, Tony Shalhoub, ...",Life or Something Like It
4,1380819,tt0128442,John Dahl,"[Crime, Drama]","[Matt Damon, Edward Norton, Gretchen Mol, John...",Rounders


On encode les colonnes genres et acteurs que sur les données d'entrainement et de test

**Les genres**

In [15]:
mlbg1 = MultiLabelBinarizer()
genres_encoded_user = mlbg1.fit_transform(df_user['genres'])
df_genres_user = pd.DataFrame(genres_encoded_user, columns=mlbg1.classes_)

    #Si on a bien le même nombre de genres que dans le dataframe d'entrainement 
if (len(liste_genres) != len(mlbg1.classes_)):
    #On récupère les genres manquants
    liste_genres_pour_user = mlbg1.classes_
    genres_manquants = set(liste_genres) - set(liste_genres_pour_user)

    #On crée un dataframe avec les colonnes de genres manquants remplies de zéros
    df_genres_manquants = pd.DataFrame(0, columns=genres_manquants, index=df_genres_user.index)

    #On concatène df_genres_user avec df_genres_manquants
    df_genres_user = pd.concat([df_genres_user, df_genres_manquants], axis=1)
    df_genres_user = df_genres_user.sort_index(axis=1)
    
    df_genres_user.head()

**Les acteurs**

In [16]:
mlba1 = MultiLabelBinarizer()
acteurs_encoded_user = mlba1.fit_transform(df_user['acteurs'])
df_acteurs_user = pd.DataFrame(acteurs_encoded_user, columns=mlba1.classes_)

if (len(liste_acteurs) != len(mlba1.classes_)):
    #On récupère les acteurs manquants
    liste_acteurs_pour_user = mlba1.classes_
    acteurs_manquants = set(liste_acteurs) - set(liste_acteurs_pour_user)

    #On crée un dataframe avec les colonnes d'acteurs manquants remplies de zéros
    df_acteurs_manquants = pd.DataFrame(0, columns=list(acteurs_manquants), index=df_acteurs_user.index)

    #On concatène df_acteurs_user avec df_acteurs_manquants
    df_acteurs_user = pd.concat([df_acteurs_user, df_acteurs_manquants], axis=1)
    df_acteurs_user = df_acteurs_user.sort_index(axis=1)
    
    df_acteurs_user.head()
    
#On concatène les DataFrames encodés avec le DataFrame d'origine
df_user = pd.concat([df_user, df_genres_user, df_acteurs_user], axis=1)

#On supprime les colonnes originales 'genres' et 'acteurs' car on en a plus besoin
df_user.drop(['genres', 'acteurs'], axis=1, inplace=True)

df_user.head()

,user_id,film,directeur,titre,Action,Adventure,Animation,Biography,Comedy,Crime,...,William H. Macy,William Hurt,William Shatner,Winona Ryder,Wood Harris,Woody Allen,Woody Harrelson,Yoshihiko Hakamada,Yun-Fat Chow,Zach Braff
0,1380819,tt0245046,Gillian Armstrong,Charlotte Gray,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1380819,tt0185125,Pedro Almodóvar,All About My Mother,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1380819,tt0308644,Marc Forster,Finding Neverland,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1380819,tt0282687,Stephen Herek,Life or Something Like It,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1380819,tt0128442,John Dahl,Rounders,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
nb_film_non_vu_par_user,_ = df_user.shape

## Encodage des données

On concatène les données pour entrainer, tester et prédire afin qu'elles utilisent le meme encodage

In [32]:
df_combined = pd.concat([df_pour_reseau.iloc[:, :-1], df_user], axis=0)

X = df_combined.values
y = df_pour_reseau['note'].values

On convertit les user_id, films, directeurs, titres en entier


In [33]:
# On s'occupe des user_id
user_id_encoder = LabelEncoder()
X[:, 0] = user_id_encoder.fit_transform(X[:, 0])

# On s'occupe des films
film_encoder = LabelEncoder()
X[:, 1] = film_encoder.fit_transform(X[:, 1])

#On s'occupe des directeurs 
directeur_encoder = LabelEncoder()
X[:, 2] = directeur_encoder.fit_transform(X[:, 2])

#On s'occupe des titres 
titre_encoder = LabelEncoder()
X[:, 3] = titre_encoder.fit_transform(X[:, 3])

In [34]:
X_user = X[-nb_film_non_vu_par_user:]
X = X[:-nb_film_non_vu_par_user]

print(X.shape)
print(X_user.shape)

(259818, 769)
(395, 769)


On divise les données en données d'apprentissages et de tests (80% pour l'apprentissage et 20% pour les test)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [36]:
X_train = X_train.astype(np.int64)
X_test = X_test.astype(np.int64)
X_user = X_user.astype(np.int64)

On convertit les ensembles de données en tensors PyTorch

In [37]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)
X_user = torch.tensor(X_user)
print(X_test.type())
print(X_train.type())
print(y_train.type())
print(y_test.type())
print(X_user.type())
y_test.shape

torch.LongTensor
torch.LongTensor
torch.LongTensor
torch.LongTensor
torch.LongTensor


torch.Size([51964])

On convertit les données d'entrées et de sorties du réseau en FloatTensor 


In [38]:
X_train = X_train.float()
X_test = X_test.float()
y_train = y_train.float()
y_test = y_test.float()
X_user = X_user.float()

print(y_train.unique())
print(y_test.unique())

tensor([1., 2., 3., 4., 5.])
tensor([1., 2., 3., 4., 5.])


## Détermination des hyperparamètres

On veut faire une régression

In [39]:
def pytorch_fit(model, X,Y, X_test, y_test, criterion, optimizer, n_epochs):
    
    torch.cuda.empty_cache()# Libération de la mémoire GPU
    train_loss = []
    test_loss = []
    
    # on switche le modèle en mode entrainement (important pour le dropout uniquement)
    model.train()

    # on parcourt "n_epochs" fois l'ensemble des donnees
    for i in range(n_epochs):

        # le "dataloader" va fournir des mini-batches de 32 exemples a la fois.
        
        dataloader = torch.utils.data.DataLoader(list(zip(X,Y)), batch_size=32, shuffle=True)

        for inputs,labels in dataloader:
            
            # Remise à zéro des gradients
            optimizer.zero_grad()

            # Calcul de la prédiction (output)
            outputs = model(inputs)

            # Calcul du coût (de la perte)
            loss = criterion(outputs, labels)

            # Calcul des gradients
            loss.backward()

            # Mise à jour des poids avec l'optimiseur
            optimizer.step()
            
            # Libération de la mémoire GPU
            torch.cuda.empty_cache()
        
        model.eval()
        
        # Calcul du cout + enregistrement pour la visualisation
        
        #J'ai un dépassement de mémoire, je calcule donc cout_train
        #uniquement sur un sous ensemble des données d'entrainements
        
        indice_ss_ensemble = np.random.choice(len(X), size=10000, replace=False)
        
        cout_train = criterion(model(X[indice_ss_ensemble]), Y[indice_ss_ensemble]).item()
        cout_test = criterion(model(X_test), y_test).item()
        train_loss.append(cout_train)
        test_loss.append(cout_test)

    # Visualiser le cout pour le trainset et le testset
    
    plt.plot(range(n_epochs), train_loss, label="train")
    plt.plot(range(n_epochs), test_loss , label="test")
    plt.xlabel('epochs')
    plt.ylabel('cost')
    plt.legend()
    plt.title('Graphique')
    plt.grid(True)
    plt.show()
    
    # on switche le modèle en mode prédiction
    model.eval()

**Evaluation du modele**

In [40]:
def RMSE(model, X, Y):
    predictions = model(X)
    rmse = torch.sqrt(F.mse_loss(predictions, Y.view(-1, 1))) #On redimensionne Y pour qu'il ait la meme forme que la prédiction
    return rmse.item()

def MAE(model, X, Y):
    predictions = model(X)
    mae = F.l1_loss(predictions, Y.view(-1, 1)) #On redimensionne Y pour qu'il ait la meme forme que la prédiction
    return mae.item()

fonction_de_cout_regression = MSELoss()

lr = taux d'apprentissage (crucial dans l'entrainement des réseaux)

Le taux d'apprentissage contrôle la taille des pas que notre algorithme d'optimisation prend lors de la mise à jour des poids du modèle
Un taux d'apprentissage trop élevé peut entraîner une divergence du modèle
Un taux d'apprentissage trop faible peut entraîner une convergence lente

n_epochs = nombre d'époques 
Le nombre d'époques spécifie combien de fois l'ensemble d'entraînement complet est utilisé pour entraîner le modèle.
Trop peu d'époques peuvent entraîner un sous-apprentissage, tandis que trop d'époques peuvent entraîner un sur-apprentissage.

In [41]:
# Création d'un dictionnaire pour stocker les réseaux de neurones avec les valeurs de taux d'apprentissage correspondantes
networks = {}

In [ ]:
for alpha in [0.01,0.001,0.0001]:
    
    # regression linéaire
    net1 = nn.Sequential(
        nn.Linear(in_features=769, out_features=1),
        nn.ReLU(),
    )

    # réseau à trois couches avec ReLU pour chaque couche
    net2 = nn.Sequential(
        nn.Linear(in_features=769, out_features=20),
        nn.ReLU(),
        nn.Linear(in_features=20, out_features=10),
        nn.ReLU(),
        nn.Linear(in_features=10, out_features=1), 
        nn.ReLU(),
    )
    
    print("Taux d'apprentissage = ",alpha)
    
    print('--Entrainement de net1 --')

    optimizer1 = Adam(net1.parameters(),lr=alpha)
    pytorch_fit(net1, X_train, y_train, X_test, y_test, fonction_de_cout_regression, optimizer1, n_epochs=50)

    print('--Entrainement de net2 --')

    optimizer2 = Adam(net2.parameters(),lr=alpha)
    pytorch_fit(net2, X_train, y_train, X_test, y_test, fonction_de_cout_regression, optimizer2, n_epochs=50)

    print("-- Fin de l'entrainement--")

    print('RMSE de net1 sur les donnees test:', RMSE(net1,X_test,y_test))
    print('MAE de net1 sur les donnees test:', MAE(net1,X_test,y_test))
    
    print('RMSE de net2 sur les donnees test:', RMSE(net2,X_test,y_test))
    print('MAE de net2 sur les donnees test:', MAE(net2,X_test,y_test))
    
    networks[alpha] = (net1, net2)

Taux d'apprentissage =  0.01
--Entrainement de net1 --


/Users/leahkalousdian/Library/Python/3.8/lib/python/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/leahkalousdian/Library/Python/3.8/lib/python/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([14])) that is different to the input size (torch.Size([14, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/leahkalousdian/Library/Python/3.8/lib/python/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])). This will likely lead to incorrect results du

Pour l'instant, le réseau donnant les meilleurs performances est le réseau net2 avec lr=0.0001 et n_epochs=50  
On essaye d'ajouter du dropout pour voir si on obtient de meilleures performances.  
On désactive aléatoirement un certain pourcentage des neurones 
lors de chaque itération de l'entraînement pour prévenir le surapprentissage.

In [ ]:
alpha = 0.0001 #le taux ayant donné les meilleures performances précédemment 

In [ ]:
net3 = nn.Sequential(
    nn.Linear(in_features=769, out_features=1),
    nn.ReLU(),
    nn.Dropout(p=0.2)  
)

net4 = nn.Sequential(
    nn.Linear(in_features=769, out_features=20),
    nn.ReLU(),
    nn.Dropout(p=0.2),  
    nn.Linear(in_features=20, out_features=10),
    nn.ReLU(),
    nn.Dropout(p=0.2), 
    nn.Linear(in_features=10, out_features=1), 
    nn.ReLU(),
)

In [ ]:
print("Taux d'apprentissage = ",alpha)

print('--Entrainement de net3 --')

optimizer3 = Adam(net3.parameters(),lr=alpha)
pytorch_fit(net3, X_train, Y_train, X_test, Y_test, fonction_de_cout_regression, optimizer3, n_epochs=50)

print('--Entrainement de net4 --')

optimizer4 = Adam(net4.parameters(),lr=alpha)
pytorch_fit(net4, X_train, Y_train, X_test, Y_test, fonction_de_cout_regression, optimizer4, n_epochs=50)

print("-- Fin de l'entrainement--")

print('RMSE de net3 sur les donnees test:', RMSE(net3,X_test,Y_test))
print('MAE de net3 sur les donnees test:', MAE(net3,X_test,Y_test))

print('RMSE de net4 sur les donnees test:', RMSE(net4,X_test,Y_test))
print('MAE de net4 sur les donnees test:', MAE(net4,X_test,Y_test))

Le fait d'ajouter du dropout sur les réseaux n'a pas l'air d'avoir un impact significatif sur les performances.   
Au vu des résulats obtenus, on a donc décidé d'utiliser le réseau net2 avec lr=0.0001 et n_epochs=50. 


## Les 10 meilleurs recommandations

On veut déterminer les 10 meilleures recommandations pour l'user_id = '1380819'.  
On a X_user qui va servir d'entrée au réseau de neurones 

On va prédire les notes pour tout les films que l'utilisateur n'a pas vu.  
On va stocker dans la liste recommandation_possible des tuples de la forme (titre du film, note)

In [ ]:
reseau = networks[0.0001][1]

recommandation_possible = [] 

reseau.eval()

for i in range(X_user.size(0)):
    input_data = X_user[i, :].unsqueeze(0)  # Nous devons ajouter une dimension pour obtenir la forme (1, features)
    prediction = net2(input_data)
    recommandation_possible.append( (df_user.iloc[i][3],prediction.item()) )

recommandation_possible.sort(key=lambda x: x[1], reverse=True)
    
if (len(recommandation_possible) < 10) : #L'utilisateur n'a pas noté moins de 10 films, il lui reste donc moins de 10 films à regarder alors on lui recommande tout les films qu'il n'a pas vu

    films_recommander = [titre for titre,note in recommandation_possible]
    print("L'utilisateur n'a pas noté moins de 10 films")
    print("Ainsi les meilleurs recommendation sont les films qu'il n'a pas vu")
    print(films_recommander)

else:
    films_recommander = [titre for titre,note in recommandation_possible[:10]]
    print("Les 10 meilleures recommandations sont : ",films_recommander)  